In [2]:
import os
import numpy as np
import mdtraj as md
import pickle
from simtk.openmm import unit
from tqdm import tqdm_notebook
from simtk.openmm import app
import matplotlib.pyplot as plt
from openeye import oechem

In [3]:
def new_positions(hybrid_positions):
    n_atoms_new = htf._topology_proposal.n_atoms_new
    hybrid_indices = [htf._new_to_hybrid_map[idx] for idx in range(n_atoms_new)]
    return hybrid_positions[hybrid_indices, :]
    
def old_positions(hybrid_positions):
    n_atoms_old = htf._topology_proposal.n_atoms_old
    hybrid_indices = [htf._old_to_hybrid_map[idx] for idx in range(n_atoms_old)]
    return hybrid_positions[hybrid_indices, :]

In [9]:
def make_traj(outdir, htf, i, phase, name, endstate, is_old=True, solvent_only=False):
    with open(os.path.join(outdir, f"{i}_{phase}_{name}_1ns_snapshots.npy"), "rb") as f:
        cache = np.load(f)
    if is_old:
        old_pos = np.zeros(shape=(100, htf._topology_proposal.old_topology.getNumAtoms(), 3))
        for j, pos in tqdm_notebook(enumerate(cache)):
            old_pos[j] = old_positions(unit.Quantity(pos, unit=unit.nanometers)).value_in_unit_system(unit.md_unit_system)
        
        traj = md.Trajectory(old_pos, md.Topology.from_openmm(htf._topology_proposal.old_topology))
    else:
        new_pos = np.zeros(shape=(100, htf._topology_proposal.new_topology.getNumAtoms(), 3))
        for j, pos in tqdm_notebook(enumerate(cache)):
            new_pos[j] = new_positions(unit.Quantity(pos, unit=unit.nanometers)).value_in_unit_system(unit.md_unit_system)
        
        traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
        
    if solvent_only:
        traj.atom_slice(traj.topology.select("water or resname 'na\+' or resn 'cl\-'"), inplace=True)
        
    # Set unit cell vectors in traj 
    box_vectors = [val.value_in_unit_system(unit.md_unit_system) for val in htf.hybrid_system.getDefaultPeriodicBoxVectors()]
    box_vectors_formatted = np.ndarray(shape=(3,3), buffer=np.array(box_vectors)).astype(np.float32) # note that mdraj expect np.ndarray, not np.arrays
    box_vectors_repeated = np.repeat(np.array([box_vectors_formatted]), traj.xyz.shape[0], axis=0)
    traj.unitcell_vectors = box_vectors_repeated
    
    traj = traj.image_molecules()
    
#     if solvent_only:
#         traj.save(os.path.join(outdir, f"{i}_{phase}_{endstate}_solvent.dcd"))
#         traj[0].save(os.path.join(outdir, f"{i}_{phase}_{endstate}_solvent.pdb"))
#     else:
#         traj.save(os.path.join(outdir, f"{i}_{phase}_{endstate}.dcd"))
#         traj[0].save(os.path.join(outdir, f"{i}_{phase}_{endstate}.pdb"))
        
    return traj
       

## 24/2 N501Y (scaled water rest)

In [5]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/24/2/"
i = 2
phase = 'complex'
endstate = 1
name = 'tyr'
is_old = False

with open(os.path.join(outdir, f"{i}_{phase}_{endstate}.pickle"), "rb") as f:
    htf = pickle.load(f)
    


INFO:rdkit:Enabling RDKit 2021.03.5 jupyter extensions


In [6]:
htf._topology_proposal.new_topology.getNumAtoms()

185384

In [10]:
traj = make_traj(outdir, htf, i, phase, name, endstate, is_old)

/scratch/lsftmp/3219225.tmpdir/ipykernel_44042/3064617031.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j, pos in tqdm_notebook(enumerate(cache)):


0it [00:00, ?it/s]

In [11]:
from simtk.openmm import app

In [16]:
app.PDBxFile.writeFile(traj.topology.to_openmm(), traj.openmm_positions(0), file=open("/data/chodera/zhangi/perses_benchmark/neq/24/2/2_complex_1.cif", "w"), keepIds=True)